In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [6]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [7]:
with open(dPath / "not_running.pkl", 'rb') as filename:
    not_running = pickle.load(filename)

In [8]:
X_train = train_data.drop("Detected", axis=1)

In [9]:
y_train = train_data.Detected

In [10]:
X_valid = valid_data.drop("Detected", axis=1)

In [11]:
y_valid = valid_data.Detected

In [12]:
with open(dPath / "rf_exp_05_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [13]:
X_train = X_train[names] 

In [14]:
X_valid = X_valid[names]

In [15]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0


In [16]:
from imblearn.over_sampling import ADASYN

In [17]:
sm = ADASYN(random_state=42, n_jobs=-1,  n_neighbors=5)

In [18]:
%time X_res, y_res = sm.fit_resample(X_train, y_train)

Wall time: 52.2 s


In [19]:
X_train_not_running = not_running.drop("Detected", axis=1)

In [20]:
y_train_not_running = not_running.Detected

In [21]:
X_train_not_running = X_train_not_running[names]

In [22]:
X_res = pd.concat([X_res,X_train_not_running], ignore_index=True,axis=0)

In [23]:
y_res = pd.concat([y_res,y_train_not_running], ignore_index=True,axis=0)

In [24]:
y_res.sum()/y_res.shape[0]

0.4748445906956071

In [52]:
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble  import HistGradientBoostingClassifier

In [102]:
hgb = HistGradientBoostingClassifier(random_state=42, max_iter=250, validation_fraction=None)

In [103]:
%time hgb.fit(X_res, y_res)

Wall time: 53.3 s


HistGradientBoostingClassifier(l2_regularization=0.0, learning_rate=0.1,
                               loss='auto', max_bins=255, max_depth=None,
                               max_iter=250, max_leaf_nodes=31,
                               min_samples_leaf=20, n_iter_no_change=None,
                               random_state=42, scoring=None, tol=1e-07,
                               validation_fraction=None, verbose=0,
                               warm_start=False)

In [104]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [105]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid), roc_auc_score(y_valid, m.predict(X_valid))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [106]:
def conf_matr(m):
    y_pred = m.predict(X_valid)
    print(classification_report(y_valid, y_pred))

In [107]:
print_score(hgb)

[0.7761745523866778, 0.679862510370985, 0.6153701293330512]


In [108]:
conf_matr(hgb)

              precision    recall  f1-score   support

       False       0.38      0.49      0.43     16657
        True       0.82      0.74      0.78     50839

    accuracy                           0.68     67496
   macro avg       0.60      0.62      0.60     67496
weighted avg       0.71      0.68      0.69     67496



In [109]:
with open(dPath / "hgb_exp_01.pkl", 'wb') as filename:
    pickle.dump(hgb,filename)